# Top Used Words
use expanded, decapitalited and punctuation removed prompts
- concatenate all prompts from all conversations per user
- then count frequencies per user
- normalize by total token count
- fuse counts within each gender group
- take top 10 or 15 words
- logistic regression or chi square to find most discriminative word

Problems:
- chi square is not interpretable

In [11]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('../../giicg.db')

prompts = pd.read_sql("SELECT * FROM expanded_prompts", conn)
prompts

,message_id,conversation_id,role,message_text,conversational,code,other,gender,user_id,language
0,1,1,user,"parsing data from python iterator, how it coul...","parsing data from python iterator, how it coul...",,,Man (cisgender),6,en
1,730,32,user,Write python function to do operations with in...,Write python function to do operations with in...,,report_dt\tsource\tmetric_name\tmetric_num\tme...,Man (cisgender),6,en
2,1133,55,user,Write shortest tutorial on creating RAG on ema...,Write shortest tutorial on creating RAG on ema...,,,Man (cisgender),6,en
3,1135,55,user,what is FAISS,what is FAISS,,,Man (cisgender),6,en
4,1137,55,user,Transform given code to process large .mbox file,Transform given code to process large .mbox file,,Transform given code to process large .mbox file,Man (cisgender),6,en
...,...,...,...,...,...,...,...,...,...,...
755,724,31,user,import pandas as pd\nimport numpy as np\nfrom ...,Please replace my retrieval pipeline here with...,import pandas as pd\nimport numpy as np\nfrom ...,You are tasked with separating user prompts in...,Man (cisgender),92,en
756,726,31,user,"please update my code accordingly, no comments...","please update my code accordingly, no comments...",,,Man (cisgender),92,en
757,1131,54,user,import pandas as pd\nimport numpy as np\nfrom ...,"I want to tune optimal thresholds. Currently, ...",import pandas as pd\nimport numpy as np\nfrom ...,The narratives list looks like this:\nnarrativ...,Man (cisgender),92,en
758,1532,71,user,"from transformers import AutoTokenizer, AutoMo...",I want to use an LLM for listwise reranking in...,"from transformers import AutoTokenizer, AutoMo...",,Man (cisgender),92,en


In [12]:
from helpers.normalization import remove_punctuation_and_newlines, remove_capitalization

prompts['conversational'] = prompts['conversational'].apply(remove_punctuation_and_newlines)
prompts['conversational'] = prompts['conversational'].apply(remove_capitalization)

prompts

,message_id,conversation_id,role,message_text,conversational,code,other,gender,user_id,language
0,1,1,user,"parsing data from python iterator, how it coul...",parsing data from python iterator how it could...,,,Man (cisgender),6,en
1,730,32,user,Write python function to do operations with in...,write python function to do operations with in...,,report_dt\tsource\tmetric_name\tmetric_num\tme...,Man (cisgender),6,en
2,1133,55,user,Write shortest tutorial on creating RAG on ema...,write shortest tutorial on creating rag on ema...,,,Man (cisgender),6,en
3,1135,55,user,what is FAISS,what is faiss,,,Man (cisgender),6,en
4,1137,55,user,Transform given code to process large .mbox file,transform given code to process large mbox file,,Transform given code to process large .mbox file,Man (cisgender),6,en
...,...,...,...,...,...,...,...,...,...,...
755,724,31,user,import pandas as pd\nimport numpy as np\nfrom ...,please replace my retrieval pipeline here with...,import pandas as pd\nimport numpy as np\nfrom ...,You are tasked with separating user prompts in...,Man (cisgender),92,en
756,726,31,user,"please update my code accordingly, no comments...",please update my code accordingly no comments ...,,,Man (cisgender),92,en
757,1131,54,user,import pandas as pd\nimport numpy as np\nfrom ...,i want to tune optimal thresholds currently i ...,import pandas as pd\nimport numpy as np\nfrom ...,The narratives list looks like this:\nnarrativ...,Man (cisgender),92,en
758,1532,71,user,"from transformers import AutoTokenizer, AutoMo...",i want to use an llm for listwise reranking in...,"from transformers import AutoTokenizer, AutoMo...",,Man (cisgender),92,en


## 1. Concatenate all prompts per user

In [13]:
user_prompts = (
    prompts.groupby(['user_id', 'gender'])['conversational']
    .apply(' '.join)
    .reset_index()    # Reset index to create a DataFrame
)

user_prompts.columns = ['user_id', 'gender', 'combined_prompts']
user_prompts = user_prompts[user_prompts['gender'].isin(['Woman (cisgender)', 'Man (cisgender)'])].reset_index()

user_prompts

,index,user_id,gender,combined_prompts
0,0,6,Man (cisgender),parsing data from python iterator how it could...
1,1,11,Woman (cisgender),can you adapt the following code so that inste...
2,2,15,Man (cisgender),setalltables action is currently not fetching ...
3,3,16,Woman (cisgender),i want to use dummy hot encoding to replace th...
4,4,25,Man (cisgender),what is the best way to encode and compress a ...
5,5,28,Woman (cisgender),i have a pandas dataframe like this i want to...
6,7,31,Man (cisgender),how can i make use of an observablehqdatabasec...
7,8,34,Man (cisgender),blender and python i have a collection of hund...
8,9,46,Man (cisgender),how to run a python future without blocking ie...
9,10,47,Man (cisgender),can you create photoshop scripts in which form...


## 1. Tokenize and count

In [14]:
import spacy
from collections import Counter

tokenizer = spacy.blank("en")

def spacy_tokenizer(text):
    #stopwords = {"a", "an", "the", "this", "that", "in", "on", "at"}  # Set of articles to remove
    #return [token.text for token in tokenizer(text) if token.text.lower() not in stopwords]
    return [token.text for token in tokenizer(text)]


def divide_counts(counts_dict, total_tokens):
    return Counter({k: v / total_tokens for k, v in counts_dict.items()})


user_prompts['tokens'] = user_prompts['combined_prompts'].apply(spacy_tokenizer)

user_prompts['counts'] = user_prompts['tokens'].apply(Counter)
user_prompts['n_tokens'] = user_prompts['tokens'].apply(lambda x: len(x))

user_prompts['normalized_counts'] = user_prompts.apply(lambda row: divide_counts(row['counts'], row['n_tokens']), axis=1)

user_prompts

,index,user_id,gender,combined_prompts,tokens,counts,n_tokens,normalized_counts
0,0,6,Man (cisgender),parsing data from python iterator how it could...,"[parsing, data, from, python, iterator, how, i...","{'parsing': 1, 'data': 2, 'from': 4, 'python':...",201,"{'parsing': 0.004975124378109453, 'data': 0.00..."
1,1,11,Woman (cisgender),can you adapt the following code so that inste...,"[can, you, adapt, the, following, code, so, th...","{'can': 9, 'you': 8, 'adapt': 3, 'the': 15, 'f...",194,"{'can': 0.04639175257731959, 'you': 0.04123711..."
2,2,15,Man (cisgender),setalltables action is currently not fetching ...,"[setalltables, action, is, currently, not, fet...","{'setalltables': 1, 'action': 1, 'is': 2, 'cur...",36,"{'setalltables': 0.027777777777777776, 'action..."
3,3,16,Woman (cisgender),i want to use dummy hot encoding to replace th...,"[i, want, to, use, dummy, hot, encoding, to, r...","{'i': 28, 'want': 6, 'to': 23, 'use': 2, 'dumm...",548,"{'i': 0.051094890510948905, 'want': 0.01094890..."
4,4,25,Man (cisgender),what is the best way to encode and compress a ...,"[what, is, the, best, way, to, encode, and, co...","{'what': 1, 'is': 6, 'the': 5, 'best': 1, 'way...",108,"{'what': 0.009259259259259259, 'is': 0.0555555..."
5,5,28,Woman (cisgender),i have a pandas dataframe like this i want to...,"[i, have, a, pandas, dataframe, like, this, ,...","{'i': 24, 'have': 7, 'a': 10, 'pandas': 1, 'da...",569,"{'i': 0.0421792618629174, 'have': 0.0123022847..."
6,7,31,Man (cisgender),how can i make use of an observablehqdatabasec...,"[how, can, i, make, use, of, an, observablehqd...","{'how': 3, 'can': 3, 'i': 3, 'make': 1, 'use':...",93,"{'how': 0.03225806451612903, 'can': 0.03225806..."
7,8,34,Man (cisgender),blender and python i have a collection of hund...,"[blender, and, python, i, have, a, collection,...","{'blender': 2, 'and': 21, 'python': 3, 'i': 26...",1323,"{'blender': 0.0015117157974300832, 'and': 0.01..."
8,9,46,Man (cisgender),how to run a python future without blocking ie...,"[how, to, run, a, python, future, without, blo...","{'how': 3, 'to': 4, 'run': 1, 'a': 4, 'python'...",69,"{'how': 0.043478260869565216, 'to': 0.05797101..."
9,10,47,Man (cisgender),can you create photoshop scripts in which form...,"[can, you, create, photoshop, scripts, in, whi...","{'can': 23, 'you': 11, 'create': 3, 'photoshop...",906,"{'can': 0.025386313465783666, 'you': 0.0121412..."


## Top 10 per User

In [15]:
def get_top_n_tokens(counter, n=10):
    # Get the n most common tokens from the counts dictionary

    return counter.most_common(n)


# Apply the function to get the top 5 tokens for each row and create a new column
user_prompts['top_10'] = user_prompts['normalized_counts'].apply(get_top_n_tokens, n=10)

# Check the updated DataFrame
user_prompts

,index,user_id,gender,combined_prompts,tokens,counts,n_tokens,normalized_counts,top_10
0,0,6,Man (cisgender),parsing data from python iterator how it could...,"[parsing, data, from, python, iterator, how, i...","{'parsing': 1, 'data': 2, 'from': 4, 'python':...",201,"{'parsing': 0.004975124378109453, 'data': 0.00...","[(to, 0.04477611940298507), (rows, 0.029850746..."
1,1,11,Woman (cisgender),can you adapt the following code so that inste...,"[can, you, adapt, the, following, code, so, th...","{'can': 9, 'you': 8, 'adapt': 3, 'the': 15, 'f...",194,"{'can': 0.04639175257731959, 'you': 0.04123711...","[(the, 0.07731958762886598), (can, 0.046391752..."
2,2,15,Man (cisgender),setalltables action is currently not fetching ...,"[setalltables, action, is, currently, not, fet...","{'setalltables': 1, 'action': 1, 'is': 2, 'cur...",36,"{'setalltables': 0.027777777777777776, 'action...","[(is, 0.05555555555555555), (why, 0.0555555555..."
3,3,16,Woman (cisgender),i want to use dummy hot encoding to replace th...,"[i, want, to, use, dummy, hot, encoding, to, r...","{'i': 28, 'want': 6, 'to': 23, 'use': 2, 'dumm...",548,"{'i': 0.051094890510948905, 'want': 0.01094890...","[(the, 0.07116788321167883), (i, 0.05109489051..."
4,4,25,Man (cisgender),what is the best way to encode and compress a ...,"[what, is, the, best, way, to, encode, and, co...","{'what': 1, 'is': 6, 'the': 5, 'best': 1, 'way...",108,"{'what': 0.009259259259259259, 'is': 0.0555555...","[(is, 0.05555555555555555), (type, 0.055555555..."
5,5,28,Woman (cisgender),i have a pandas dataframe like this i want to...,"[i, have, a, pandas, dataframe, like, this, ,...","{'i': 24, 'have': 7, 'a': 10, 'pandas': 1, 'da...",569,"{'i': 0.0421792618629174, 'have': 0.0123022847...","[(the, 0.07205623901581722), (i, 0.04217926186..."
6,7,31,Man (cisgender),how can i make use of an observablehqdatabasec...,"[how, can, i, make, use, of, an, observablehqd...","{'how': 3, 'can': 3, 'i': 3, 'make': 1, 'use':...",93,"{'how': 0.03225806451612903, 'can': 0.03225806...","[(of, 0.043010752688172046), (a, 0.04301075268..."
7,8,34,Man (cisgender),blender and python i have a collection of hund...,"[blender, and, python, i, have, a, collection,...","{'blender': 2, 'and': 21, 'python': 3, 'i': 26...",1323,"{'blender': 0.0015117157974300832, 'and': 0.01...","[(the, 0.07634164777021919), (a, 0.02947845804..."
8,9,46,Man (cisgender),how to run a python future without blocking ie...,"[how, to, run, a, python, future, without, blo...","{'how': 3, 'to': 4, 'run': 1, 'a': 4, 'python'...",69,"{'how': 0.043478260869565216, 'to': 0.05797101...","[(to, 0.057971014492753624), (a, 0.05797101449..."
9,10,47,Man (cisgender),can you create photoshop scripts in which form...,"[can, you, create, photoshop, scripts, in, whi...","{'can': 23, 'you': 11, 'create': 3, 'photoshop...",906,"{'can': 0.025386313465783666, 'you': 0.0121412...","[(the, 0.09161147902869757), (a, 0.02869757174..."


## Top 10 per Gender

In [16]:
#filtered_data = user_prompts[user_prompts['gender'].isin(['Woman (cisgender)', 'Man (cisgender)'])]


gender_aggregated_counts = {}
for gender, group in user_prompts.groupby('gender'):
    # Start with an empty Counter and update it with counts from each row
    combined_counter = Counter()
    for row_counter in group['normalized_counts']:
        combined_counter.update(row_counter)
    gender_aggregated_counts[gender] = combined_counter

# Function to get top N tokens from a Counter
def get_top_n_from_counter(counter, n=10):
    return counter.most_common(n)

# Get the top N tokens for each gender
top_n_per_gender = {
    gender: get_top_n_from_counter(counter, n=10)
    for gender, counter in gender_aggregated_counts.items()
}

# Display the results
for gender, top_tokens in top_n_per_gender.items():
    print(f"Top tokens for {gender.capitalize()}:")
    for token, freq in top_tokens:
        print(f"{token}: {freq:.4f}")
    print()


Top tokens for Man (cisgender):
the: 0.9192
to: 0.4788
a: 0.4456
i: 0.4297
is: 0.3503
of: 0.2701
in: 0.2485
and: 0.2343
it: 0.1942
that: 0.1903

Top tokens for Woman (cisgender):
the: 0.7127
i: 0.4386
to: 0.3586
a: 0.3151
and: 0.2610
can: 0.2483
you: 0.1992
of: 0.1933
in: 0.1727
code: 0.1709



In [17]:
import numpy as np
from scipy.stats import chisquare

# Combine frequencies for unified vocab
unified_vocab = set(gender_aggregated_counts['Man (cisgender)']) | set(gender_aggregated_counts['Woman (cisgender)'])
#print(unified_vocab)
#print(gender_aggregated_counts["Man (cisgender)"])
results = {}

for word in unified_vocab:
    male_count = gender_aggregated_counts["Man (cisgender)"].get(word, 0)
    female_count = gender_aggregated_counts["Woman (cisgender)"].get(word, 0)
    
    observed = [male_count, female_count]
    total = sum(observed)
    expected = [total / 2, total / 2]  # Equal division under null hypothesis

    # Perform chi-square test
    chi2, p = chisquare(f_obs=observed, f_exp=expected)
    results[word] = {"chi2": chi2, "p_value": p}

# Sort words by discriminative power (lower p-value, higher chi2)
discriminative_words = sorted(results.items(), key=lambda x: x[1]['p_value'])

#print(discriminative_words)

# Filter by p-value < 0.05 and sort by chi2 in descending order
filtered_discriminative_words = [
    (word, values["chi2"], values["p_value"])
    for word, values in results.items()
    if values["p_value"] < 0.05
]
filtered_discriminative_words.sort(key=lambda x: x[1], reverse=True)

# Print each word with chi2 and p-value in one line
for word, stats in discriminative_words:
    print(f"Word: {word}, Chi2: {stats['chi2']:.4f}, P-value: {stats['p_value']:.4f}")




Word: 	, Chi2: 0.1144, P-value: 0.7352
Word: is, Chi2: 0.0744, P-value: 0.7851
Word: you, Chi2: 0.0723, P-value: 0.7881
Word: type, Chi2: 0.0514, P-value: 0.8206
Word: column, Chi2: 0.0450, P-value: 0.8320
Word: me, Chi2: 0.0431, P-value: 0.8355
Word: if, Chi2: 0.0409, P-value: 0.8398
Word: from, Chi2: 0.0406, P-value: 0.8403
Word: fetching, Chi2: 0.0394, P-value: 0.8427
Word: please, Chi2: 0.0391, P-value: 0.8433
Word: why, Chi2: 0.0387, P-value: 0.8441
Word: currently, Chi2: 0.0354, P-value: 0.8508
Word: gif, Chi2: 0.0345, P-value: 0.8527
Word: snippet, Chi2: 0.0345, P-value: 0.8527
Word: scenes, Chi2: 0.0345, P-value: 0.8527
Word: videos, Chi2: 0.0345, P-value: 0.8527
Word: matching, Chi2: 0.0345, P-value: 0.8527
Word: new, Chi2: 0.0327, P-value: 0.8566
Word: n, Chi2: 0.0325, P-value: 0.8570
Word: give, Chi2: 0.0317, P-value: 0.8587
Word: function, Chi2: 0.0308, P-value: 0.8607
Word: without, Chi2: 0.0306, P-value: 0.8611
Word: iframe, Chi2: 0.0303, P-value: 0.8618
Word: distributio

## Logistic Regression

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Vectorize the prompts to create a document-term matrix
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(user_prompts['combined_prompts'])

# Convert to a DataFrame for better interpretability
word_frequencies = pd.DataFrame(
    X.toarray(),
    columns=vectorizer.get_feature_names_out()
)

# Add gender as the target variable
word_frequencies['gender'] = user_prompts['gender'].apply(lambda x: 1 if x == 'Woman (cisgender)' else 0)


# Separate features (X) and target (y)
X = word_frequencies.drop('gender', axis=1)
y = word_frequencies['gender']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Initialize the logistic regression model
log_reg = LogisticRegression(max_iter=1000)

# Train the model
log_reg.fit(X_train, y_train)


# Get feature names and coefficients
coefficients = log_reg.coef_[0]
words = vectorizer.get_feature_names_out()

# Create a DataFrame for easier interpretation
coeff_df = pd.DataFrame({
    'word': words,
    'coefficient': coefficients
})

# Sort by absolute value of coefficients for most discriminative words
discriminative_words = coeff_df.sort_values(by='coefficient', ascending=False)

# Top words indicating 'Woman (cisgender)'
top_women_words = discriminative_words.head(10)

# Top words indicating 'Man (cisgender)'
top_men_words = discriminative_words.tail(10)


print(top_women_words)
print(top_men_words)

# Make predictions
y_pred = log_reg.predict(X_test)

# Generate a classification report
report = classification_report(y_test, y_pred)
print(report)




         word  coefficient
2099      you     0.326493
300       can     0.287037
1192      new     0.239750
2063  without     0.201786
134       and     0.185154
1109       me     0.178489
175        as     0.161394
1374   please     0.154873
319    change     0.142488
370      code     0.139022
              word  coefficient
562   distribution    -0.095904
478      currently    -0.096685
128             an    -0.104322
1213           not    -0.104907
1851           the    -0.113204
782       function    -0.123949
1237            of    -0.129628
893             if    -0.139156
1976           use    -0.145196
961             is    -0.264371
              precision    recall  f1-score   support

           0       0.67      1.00      0.80         4
           1       0.00      0.00      0.00         2

    accuracy                           0.67         6
   macro avg       0.33      0.50      0.40         6
weighted avg       0.44      0.67      0.53         6



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beha